# INITIALIZATION

- **mount GoogleDrive**

In [0]:
from google.colab import drive
drive.mount('GoogleDrive')


- **unmount GoogleDrive**

In [0]:
! fusermount -u 'GoogleDrive'

# Code

In [0]:
#@title simple_cnn { display-mode: "both" }
# conding: utf-8
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
# import tensorflow_datasets as tfds
import numpy as np
import os

In [0]:
#@title parameter settings { display-mode: "both" }

tf.logging.set_verbosity(tf.logging.ERROR)

learning_rate_or = 3e-3 #@param {type: "number"}
scale_factor = 60000 #@param {type: "number"}, scale factor of l2 regularization, default is 3000

batchsize = 128 #@param {type: "integer"}, batch size, default is 128
num_epochs = 6000 #@param {type: "integer"}

event_path = 'GoogleDrive/My Drive/Colab Notebooks/Tensorboard' #@param {type: "string"}
checkpoint_path = 'GoogleDrive/My Drive/Colab Notebooks/Checkpoints' #@param {type: "string"}
online_test = True #@param {type: "boolean"}


In [4]:
#@title loading data { display-mode: "both" }
filepath = 'sample_data/MNIST_data'
mnist = input_data.read_data_sets(filepath, one_hot=False)
imgs_train, labels_train = mnist.train.images, mnist.train.labels
imgs_test, labels_test = mnist.test.images, mnist.test.labels
num_samples = labels_test.shape[0]
assert imgs_train.max() == 1., "warning: 'The value of the pixel is preferably between 0 and 1.'"

Extracting sample_data/MNIST_data/train-images-idx3-ubyte.gz
Extracting sample_data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting sample_data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting sample_data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
#@title main programm { display-mode: "both" }
#@markdown - build the graph
graph = tf.Graph()
with graph.as_default():
    
    # Exponential decay learning_rate, attenuation factor 0.9, attenuation step number 100
    global_step = tf.Variable(0, name='global_step', trainable=False)
    decay_steps = 400
    decay_rate = 0.90
    learning_rate = tf.train.exponential_decay(learning_rate_or,
                                                global_step=global_step,
                                                decay_steps=decay_steps,
                                                decay_rate=decay_rate,
                                                staircase=True,
                                                name='learning_rate')
    
    # 
    x_p = tf.placeholder(tf.float32, shape=[None, 784], name='input_images')
    y_p = tf.placeholder(tf.int64, shape=[None, ], name='labels')
    batch_size = tf.placeholder(tf.int64, name='batch_size')
    # is_training = tf.placeholder(tf.bool, name='is_training')
    data = tf.data.Dataset.from_tensor_slices((x_p, y_p))
    # if is_training is not None:
    #     data = data.repeat()
    #     data_batch = data.shuffle(2).batch(FLAGS.batch_size).prefetch(FLAGS.batch_size)
    # else:
    #     num_samples = x_p.get_shape().as_list()[0]
    #     data_batch = data.batch(num_samples).prefetch(num_samples)
    data = data.repeat()
    data_batch = data.shuffle(2).batch(batch_size).prefetch(batch_size)
    iterator = data_batch.make_initializable_iterator()
    
    # ------------------input layer-----------------
    with tf.name_scope('Input'):
        x_input, y_input = iterator.get_next()
        y = tf.one_hot(y_input, depth=10)
        keep_pro = tf.placeholder(tf.float32)
        x_imgs = tf.reshape(x_input, shape=[-1, 28, 28, 1], name='input_images')
    
    # ------------------conv1 layer-----------------
    with tf.name_scope('Conv1'):
        w_1 = tf.Variable(tf.truncated_normal([3, 3, 1, 32], stddev=0.1), name='weights_conv1')
        b_1 = tf.Variable(tf.constant(0.1, shape=[32]), name='bias_conv1')
        h_conv1 = tf.nn.relu(tf.nn.conv2d(x_imgs, w_1, strides=[1, 1, 1, 1], padding='SAME') + b_1)
        h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # ------------------conv2 layer-----------------
    with tf.name_scope('Conv2'):
        w_2 = tf.Variable(tf.truncated_normal([3, 3, 32, 64], stddev=0.1), name='weights_conv2')
        b_2 = tf.Variable(tf.constant(0.1, shape=[64]), name='bias_conv2')
        h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, w_2, strides=[1, 1, 1, 1], padding='SAME') + b_2)
        h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        # layer_shape = h_pool2.get_shape().as_list()
        # num_f = reduce(lambda a,b:a * b, layer_shape[1:])
        # h_pool2_fla = tf.reshape(h_pool2, shape=[-1, num_f])
    
    # ------------------fc1 layer-----------------
    with tf.name_scope('FC1'):
        h_pool2_fla = tf.layers.flatten(h_pool2)
        num_f = h_pool2_fla.get_shape().as_list()[-1]
    
        w_fc1 = tf.Variable(tf.truncated_normal([num_f, 128], stddev=0.1), name='weights_fc1')
        b_fc1 = tf.Variable(tf.constant(0.1, shape=[128]), name='bias_fc1')
        h_fc1 = tf.nn.relu(tf.matmul(h_pool2_fla, w_fc1) + b_fc1)
        h_drop1 = tf.nn.dropout(h_fc1, keep_prob=keep_pro, name='Dropout')
    
    # ------------------output layer-----------------
    with tf.name_scope('Output'):
        w_fc2 = tf.Variable(tf.truncated_normal([128, 10], stddev=0.1), name='weights_fc2')
        b_fc2 = tf.Variable(tf.constant(0.1, shape=[10]), name='bias_fc2')
        h_fc2 = tf.matmul(h_drop1, w_fc2) + b_fc2

#@markdown - use l2 regularization
    # ---------------------regularization_L2----------------
    tf.add_to_collection(tf.GraphKeys.WEIGHTS, w_fc1)
    tf.add_to_collection(tf.GraphKeys.WEIGHTS, w_fc2)
    regularizer = tf.contrib.layers.l2_regularizer(scale=15./scale_factor)
    reg_tem = tf.contrib.layers.apply_regularization(regularizer)
    with tf.name_scope('loss'):
        entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=h_fc2))
        entropy_loss = tf.add(entropy_loss, reg_tem, name='l2_loss')
    
    with tf.name_scope('accuracy'):
        prediction = tf.cast(tf.equal(tf.arg_max(h_fc2, 1), tf.argmax(y, 1)), "float")
        accuracy = tf.reduce_mean(prediction)
    
    with tf.name_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(entropy_loss, global_step=global_step)
    
    # ----------------------summaries-----------------------
    tf.summary.image('input_images', x_imgs, max_outputs=3, collections=['train', 'test'])
    tf.summary.histogram('conv1_weights', w_1, collections=['train'])
    tf.summary.histogram('conv1_bias', b_1, collections=['train'])
    tf.summary.scalar('loss', entropy_loss, collections=['train', 'test'])
    tf.summary.scalar('accuracy', accuracy, collections=['train', 'test'])
    tf.summary.scalar('global_step', global_step, collections=['train'])
    tf.summary.scalar('learning_rate', learning_rate, collections=['train'])
    summ_train = tf.summary.merge_all('train')
    summ_test = tf.summary.merge_all('test')
    
    #@markdown - start the session
    max_acc = 100.
    min_loss = 0.1
    sess = tf.Session(graph=graph)
    with sess.as_default():
        sess.run(tf.global_variables_initializer())
        sess.run(iterator.initializer, feed_dict={x_p: imgs_train, y_p: labels_train, batch_size: batchsize})
        summ_train_dir = os.path.join(event_path, 'summaries','train')
        summ_train_Writer = tf.summary.FileWriter(summ_train_dir)
        summ_train_Writer.add_graph(sess.graph)
        summ_test_dir = os.path.join(event_path, 'summaries', 'test')
        summ_test_Writer = tf.summary.FileWriter(summ_test_dir)
        summ_test_Writer.add_graph(sess.graph)
        saver = tf.train.Saver(max_to_keep=1)
        print(' Training ========== (。・`ω´・) ========')
        for num in range(num_epochs):
            _, acc, loss, rs = sess.run([train_op, accuracy, entropy_loss, summ_train], feed_dict={keep_pro: 0.5, 
                                                                                                    batch_size: batchsize})
            summ_train_Writer.add_summary(rs, global_step=num)
            acc *= 100
            num_e = str(num + 1)
            print_list = [num_e, loss, acc]
            if (num + 1) % 400 == 0:
                print('Keep on training ========== (。・`ω´・) ========')
                # print(b_x.shape)
                # print(b_t)
                print('Epoch {0[0]}, train_loss is {0[1]:.4f}, accuracy is {0[2]:.2f}%.\n'.format(print_list))
                
                if online_test:
                    print(' '*12, 'Online-Testing ========== (。・`ω´・) ========')
                    sess.run(iterator.initializer, feed_dict={x_p: imgs_test, y_p: labels_test, batch_size: num_samples})
                    acc, loss, rs = sess.run([accuracy, entropy_loss, summ_test], feed_dict={keep_pro: 1., 
                                                                                                batch_size: num_samples})
                    summ_test_Writer.add_summary(rs, global_step=num)
                    acc *= 100
                    print_list = [num_e, loss, acc]
                    # print(' '*10, b_x.shape)
                    # print(' '*10, b_t)
                    print(' '*10, 'Epoch {0[0]}, test_loss is {0[1]:.4f}, accuracy is {0[2]:.2f}%.\n'.format(print_list))
                    
                    sess.run(iterator.initializer, feed_dict={x_p: imgs_train, y_p: labels_train, batch_size: batchsize})
    print('\n', 'Training completed.')
    summ_train_Writer.close()
    summ_test_Writer.close()
    sess.close()

 Training ========== (。・`ω´・) ========
Keep on training ========== (。・`ω´・) ========
Epoch 400, train_loss is 0.2496, accuracy is 93.75%.

             Online-Testing ========== (。・`ω´・) ========
           Epoch 400, test_loss is 0.1912, accuracy is 97.05%.

Keep on training ========== (。・`ω´・) ========
Epoch 800, train_loss is 0.1336, accuracy is 99.22%.

             Online-Testing ========== (。・`ω´・) ========
           Epoch 800, test_loss is 0.1204, accuracy is 98.28%.

Keep on training ========== (。・`ω´・) ========
Epoch 1200, train_loss is 0.1004, accuracy is 99.22%.

             Online-Testing ========== (。・`ω´・) ========
           Epoch 1200, test_loss is 0.0966, accuracy is 98.67%.

Keep on training ========== (。・`ω´・) ========
Epoch 1600, train_loss is 0.0805, accuracy is 99.22%.

             Online-Testing ========== (。・`ω´・) ========
           Epoch 1600, test_loss is 0.0872, accuracy is 98.73%.

Keep on training ========== (。・`ω´・) ========
Epoch 2000, train_loss is 0

In [0]:
! fusermount -u 'GoogleDrive'